In [1]:
!unzip "/content/drive/MyDrive/UROP/COMP4901V Project/archive.zip"

串流輸出內容已截斷至最後 5000 行。
  inflating: train/images/000000275742.jpg  
  inflating: train/images/000000275743.jpg  
  inflating: train/images/000000275744.jpg  
  inflating: train/images/000000275745.jpg  
  inflating: train/images/000000275746.jpg  
  inflating: train/images/000000275747.jpg  
  inflating: train/images/000000275748.jpg  
  inflating: train/images/000000275749.jpg  
  inflating: train/images/000000275750.jpg  
  inflating: train/images/000000275751.jpg  
  inflating: train/images/000000275752.jpg  
  inflating: train/images/000000275753.jpg  
  inflating: train/images/000000275754.jpg  
  inflating: train/images/000000275755.jpg  
  inflating: train/images/000000275756.jpg  
  inflating: train/images/000000275757.jpg  
  inflating: train/images/000000275758.jpg  
  inflating: train/images/000000275759.jpg  
  inflating: train/images/000000275760.jpg  
  inflating: train/images/000000275761.jpg  
  inflating: train/images/000000275762.jpg  
  inflating: train/images/00000027

In [2]:
import json
with open('/content/annotation.json', 'r') as f:
    annot_data = json.load(f)

In [3]:
import pandas as pd
image_df = pd.DataFrame(annot_data['images'])

In [4]:
annot_df = pd.DataFrame(annot_data['annotations'])

In [5]:
len(annot_df)

2395553

In [ ]:
!pip install pycocotools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!unzip "/content/drive/MyDrive/UROP/COMP4901V Project/building_height_dataset_128.zip"

串流輸出內容已截斷至最後 5000 行。
  inflating: GT/NYC_68656_24080.json  
  inflating: GT/NYC_46592_39872.json  
  inflating: GT/NJ_33040_3136.json   
  inflating: GT/NYC_49168_9856.json  
  inflating: GT/NYC_34160_43680.json  
  inflating: GT/NYC_39872_40880.json  
  inflating: GT/NYC_56000_19376.json  
  inflating: GT/NYC_51184_6608.json  
  inflating: GT/NJ_19936_20608.json  
  inflating: GT/NYC_53312_30688.json  
  inflating: GT/NYC_11648_23184.json  
  inflating: GT/NYC_44240_12656.json  
  inflating: GT/NYC_62608_6048.json  
  inflating: GT/NYC_62832_20272.json  
  inflating: GT/NYC_62608_23968.json  
  inflating: GT/NYC_71232_33936.json  
  inflating: GT/NYC_36400_36736.json  
  inflating: GT/NYC_46256_27328.json  
  inflating: GT/NYC_29904_39200.json  
  inflating: GT/NYC_31920_44688.json  
  inflating: GT/NYC_72016_32256.json  
  inflating: GT/NYC_39536_30464.json  
  inflating: GT/NYC_37744_12320.json  
  inflating: GT/NYC_17808_16128.json  
  inflating: GT/NYC_68432_19936.json  
  inflati

In [ ]:
import os
import json
from PIL import Image
from pycocotools.coco import COCO
from sklearn.model_selection import train_test_split

# Path to the directory containing the images
image_dir = 'train'
files = os.listdir(image_dir)
# Split the files into training and validation sets
train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)

In [ ]:
# Path to the JSON file containing the annotations
json_dir = 'GT'

# Path to the output COCO JSON file
output_file = 'coco_train.json'

# Initialize the COCO object
coco = COCO()

# Add categories to the COCO object
categories = [{'id': 1, 'name': 'building'}]  # Change the name and ID as needed
coco.dataset['categories'] = categories
coco.createIndex()
coco.dataset['images'] = []
coco.dataset['annotations'] = []

sorted_files = sorted(train_files, key=lambda x: os.path.splitext(x)[0])
max_n = 0
mean_n = 0
image_id = 1
ann_id = 1
for file in sorted_files:
    json_file = os.path.join(json_dir, file[:-4]+'.json')
    # Load the annotations from the JSON file
    with open(json_file, 'r') as f:
        annotations = json.load(f)
        
        if max_n < len(annotations):
            max_n = len(annotations)
        mean_n = mean_n + len(annotations)
        for annotation_id in annotations:
            annotation = annotations[annotation_id]
            image_path = os.path.join(image_dir, file)
            width, height = (128,128)
            # Add the image to the COCO object
            image_dict = {'file_name': file, 'height': height, 'width': width, 'id': image_id}
            coco.dataset['images'].append(image_dict)
            coco.imgs[str(image_id)] = image_dict

            # Add the annotation to the COCO object
            category_id = 1  # Change the ID as needed
            bbox = [annotation['x'], annotation['y'], annotation['w'], annotation['h']]
            area = annotation['w'] * annotation['h']
            z = annotation['z']
            segmentation = []  # Change as needed
            iscrowd = 0  # Change as needed
            annotation_dict = {'id': ann_id, 'image_id': image_id, 'category_id': category_id, 'bbox': bbox,
                            'area': area, 'segmentation': segmentation, 'iscrowd': iscrowd, 'z': z}
            coco.dataset['annotations'].append(annotation_dict)
            coco.imgToAnns[str(image_id)].append(annotation_dict)
            coco.anns[str(ann_id)] = annotation_dict
            ann_id += 1

    image_id += 1

print("The maximum number of objects: " + str(max_n))
print("The mean number of objects: " + str(mean_n/len(sorted_files)))
# Save the COCO dataset to a JSON file
with open(output_file, 'w') as f:
    json.dump(coco.dataset, f)

creating index...
index created!
The maximum number of objects: 39
The mean number of objects: 6.981247347037778


In [ ]:
import os
import json
from PIL import Image
from pycocotools.coco import COCO
from sklearn.model_selection import train_test_split

# Path to the JSON file containing the annotations
json_dir = 'GT'

# Path to the output COCO JSON file
output_file = 'coco_val.json'

# Initialize the COCO object
coco = COCO()

# Add categories to the COCO object
categories = [{'id': 1, 'name': 'building'}]  # Change the name and ID as needed
coco.dataset['categories'] = categories
coco.createIndex()
coco.dataset['images'] = []
coco.dataset['annotations'] = []
max_n = 0
sorted_files = sorted(val_files, key=lambda x: os.path.splitext(x)[0])

for file in sorted_files:
    json_file = os.path.join(json_dir, file[:-4]+'.json')
    # Load the annotations from the JSON file
    with open(json_file, 'r') as f:
        annotations = json.load(f)
        
        for annotation_id in annotations:
            annotation = annotations[annotation_id]
            image_path = os.path.join(image_dir, file)
            width, height = (128,128)

            # Add the image to the COCO object
            image_dict = {'file_name': file, 'height': height, 'width': width, 'id': image_id}
            coco.dataset['images'].append(image_dict)
            coco.imgs[str(image_id)] = image_dict

            if max_n < len(annotations):
                max_n = len(annotations)

            # Add the annotation to the COCO object
            category_id = 1  # Change the ID as needed
            bbox = [annotation['x'], annotation['y'], annotation['w'], annotation['h']]
            area = annotation['w'] * annotation['h']
            z = annotation['z']
            segmentation = []  # Change as needed
            iscrowd = 0  # Change as needed
            annotation_dict = {'id': ann_id, 'image_id': image_id, 'category_id': category_id, 'bbox': bbox,
                            'area': area, 'segmentation': segmentation, 'iscrowd': iscrowd, 'z': z}
            coco.dataset['annotations'].append(annotation_dict)
            coco.imgToAnns[str(image_id)].append(annotation_dict)
            coco.anns[str(ann_id)] = annotation_dict
            ann_id += 1

    image_id += 1

print("The maximum number of objects: " + str(max_n))
# Save the COCO dataset to a JSON file
with open(output_file, 'w') as f:
    json.dump(coco.dataset, f)

creating index...
index created!
The maximum number of objects: 37


In [ ]:
!zip -r COCO_dataset_128.zip image coco_train.json coco_val.json

串流輸出內容已截斷至最後 5000 行。
  adding: image/NYC_43568_37520.png (deflated 2%)
  adding: image/NYC_33600_21392.png (deflated 0%)
  adding: image/NYC_49392_14336.png (deflated 2%)
  adding: image/NYC_53424_47600.png (deflated 2%)
  adding: image/NYC_55776_4144.png (deflated 3%)
  adding: image/NYC_48160_38640.png (deflated 1%)
  adding: image/NYC_22960_19600.png (deflated 1%)
  adding: image/NYC_31472_49504.png (deflated 1%)
  adding: image/NYC_60928_18928.png (deflated 3%)
  adding: image/NYC_36288_32032.png (deflated 1%)
  adding: image/NJ_22512_16464.png (deflated 3%)
  adding: image/NYC_52304_17136.png (deflated 1%)
  adding: image/NYC_50288_6048.png (deflated 3%)
  adding: image/NYC_59920_8736.png (deflated 2%)
  adding: image/NYC_19488_33936.png (deflated 1%)
  adding: image/NYC_33376_50736.png (deflated 3%)
  adding: image/NYC_17584_24416.png (deflated 1%)
  adding: image/NYC_35392_25536.png (deflated 4%)
  adding: image/NYC_57232_18704.png (deflated 3%)
  adding: image/NYC_35280_50176.p